<a href="https://colab.research.google.com/github/Gargi1971/NITK-Internship/blob/main/ResNet18_for_OrganMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install medmnist
import medmnist

from medmnist import OrganMNIST3D

In [9]:
%cd /content/drive/My Drive/MedMNIST

train_dataset = medmnist.OrganMNIST3D(root='./', download=True, split= 'train' )
test_dataset = medmnist.OrganMNIST3D(root='./', download=True, split= 'test')

/content/drive/My Drive/MedMNIST
Using downloaded and verified file: ./organmnist3d.npz
Using downloaded and verified file: ./organmnist3d.npz


In [10]:
import numpy as np
dataset = np.load('/content/drive/My Drive/MedMNIST/organmnist3d.npz')

In [ ]:
print(dataset.files)
print(dataset['train_images'])

In [ ]:
train_images_shape = dataset['train_images'].shape
print(train_images_shape)


(971, 28, 28, 28)


In [11]:
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv3d(
            in_planes, planes, kernel_size=(3, 3, 3), stride=stride, padding=(1, 1, 1), bias=False)
        self.bn1 = nn.BatchNorm3d(planes)

        self.conv2 = nn.Conv3d(planes, planes, kernel_size=(3, 3, 3),
                               stride=1, padding=(1, 1, 1), bias=False)
        self.bn2 = nn.BatchNorm3d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv3d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm3d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv3d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm3d(planes)
        self.conv2 = nn.Conv3d(planes, planes, kernel_size=(3, 3, 3),
                               stride=stride, padding=(1, 1, 1), bias=False)
        self.bn2 = nn.BatchNorm3d(planes)
        self.conv3 = nn.Conv3d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm3d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv3d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm3d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, in_channels=1, num_classes=2):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv3d(in_channels, 64, kernel_size=(3, 3, 3),
                               stride=1, padding=(1, 1, 1), bias=False)
        self.bn1 = nn.BatchNorm3d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18(in_channels, num_classes):
    return ResNet(BasicBlock, [2, 2, 2, 2], in_channels=in_channels, num_classes=num_classes)


In [12]:
!pip install torchsummary
from torchsummary import summary

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from sklearn.metrics import precision_score, recall_score


train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet18(in_channels=1, num_classes=11).to(torch.float64).to(device)

model.to(torch.float32)
sample_input = torch.randn(1, 28, 28, 28)
summary(model, input_size=sample_input.size())

for param in model.parameters():
    # Print the shape of the weights
    print(param.shape)





----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1       [-1, 64, 28, 28, 28]           1,728
       BatchNorm3d-2       [-1, 64, 28, 28, 28]             128
            Conv3d-3       [-1, 64, 28, 28, 28]         110,592
       BatchNorm3d-4       [-1, 64, 28, 28, 28]             128
            Conv3d-5       [-1, 64, 28, 28, 28]         110,592
       BatchNorm3d-6       [-1, 64, 28, 28, 28]             128
        BasicBlock-7       [-1, 64, 28, 28, 28]               0
            Conv3d-8       [-1, 64, 28, 28, 28]         110,592
       BatchNorm3d-9       [-1, 64, 28, 28, 28]             128
           Conv3d-10       [-1, 64, 28, 28, 28]         110,592
      BatchNorm3d-11       [-1, 64, 28, 28, 28]             128
       BasicBlock-12       [-1, 64, 28, 28, 28]               0
           Conv3d-13      [-1, 128, 14, 14, 14]         221,184
      BatchNorm3d-14      [-1, 128, 14,

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from sklearn.metrics import precision_score, recall_score


train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet18(in_channels=1, num_classes=11).to(torch.float64).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


num_epochs = 100
for epoch in range(num_epochs):
    model.train()  # Set the model to train mode
    running_loss = 0.0
    total_correct = 0
    total_samples = 0
    for i, (inputs, labels) in enumerate (train_loader):
        inputs, labels = inputs.to(device), labels.to(device).view(-1)
        optimizer.zero_grad()
        outputs = model(inputs)
        reg = sum(torch.sum(p**2) for p in model.parameters() if p.requires_grad)
        lambda_ = 0.0001
        loss = criterion(outputs, labels) + (lambda_ / 2) * reg
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

        _, predicted = torch.max(outputs, 1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

        print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

    epoch_loss = running_loss / len(train_dataset)
    print(f"Training Loss: {epoch_loss:.4f}")

    training_accuracy = total_correct / total_samples
    print(f"Training Accuracy: {training_accuracy:.4f}")

    # Validation loop
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    predicted_labels = []
    true_labels = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device).view(-1)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            predicted_labels.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    val_acc = correct / total
    print(f"Validation Accuracy: {val_acc:.4f}")

    precision = precision_score(true_labels, predicted_labels, average='weighted')
    recall = recall_score(true_labels, predicted_labels, average='weighted')
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}")




KeyboardInterrupt: 